In [1]:
import os
import sys
from datetime import datetime
import helper as hp
import math
import numpy as np
import time
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [2]:
hp.FIRED_BASE_FOLDER = "<Set directory of FIRED>"

startDate = "2020.07.02" # None to select dataset first day
stopDate = "2020.07.03" # None to select dataset last day (timepoint) 

# Get recording range from files and args 
start, end = hp.getRecordingRange(startDate, stopDate)
duration = int(end - start)

print(hp.time_format_ymdhms(start) + "->" + hp.time_format_ymdhms(end))

measure = "p" # p,q,s
smoothing = 3

FileNotFoundError: [Errno 2] No such file or directory: '<Set directory of FIRED>/summary/50Hz'

In [3]:
# Get mapping from powermerter to appliance
deviceMapping = hp.getDeviceMapping()
deviceInfo = hp.getDeviceInfo()
meterList = hp.getMeterList()
smartMeterName = hp.getSmartMeter()

FileNotFoundError: [Errno 2] No such file or directory: '<Set directory of FIRED>/info/deviceMapping.json'

In [4]:
dataList = [hp.getMeterPower(meter, 1, startTs=start, stopTs=end) for meter in meterList]
smData = hp.getMeterPower(smartMeterName, 1, startTs=start, stopTs=end)
# Prepare smartmeter data
dt = [(m, '<f4') for m in ["p","q","s"]]
data = np.recarray((len(smData["data"]),), dtype=dt).view(np.recarray)
for m in data.dtype.names: data[m][:] = 0
for i in [1,2,3]: 
    for m in data.dtype.names: data[m] += smData["data"][m + "_l" + str(i)]
smData["data"] = data
smData["phase"] = 1
dataList.append(smData)

# List with all appliances in loaded data
appliances = []
for dev in dataList: appliances.extend(deviceMapping[dev["title"]]["appliances"])

NameError: name 'meterList' is not defined

Load Lights data

In [5]:
lightsData = hp.getPowerForLights(1, start, end)
for light in lightsData: 
    light["title"] = light["name"]
    del light["csvData"]
    appliances.append(light["title"])
dataList.extend(lightsData)

NameError: name 'start' is not defined

Estimate Base power consumption

In [6]:
baseData = hp.getBasePower(1, start, end)
base = baseData[0]
for b in baseData[1:]: 
    for m in base["data"].dtype.names: base["data"][m] += b["data"][m]
dataList.append(base)

NameError: name 'start' is not defined

Load Stove power consumption

In [7]:
stoveLX = hp.getPowerStove(1, start, end)
stove = stoveLX[0]
for b in stoveLX[1:]: 
    for m in stove["data"].dtype.names: stove["data"][m] += b["data"][m]
dataList.append(stove)
appliances.append("stove")

NameError: name 'start' is not defined

In [8]:
meterData = []
for d in dataList:
    for m in d["data"].dtype.names: d["data"][m] = np.abs(d["data"][m])
    d["avgPower"] = np.mean(d["data"][measure])
    if smartMeterName == d["title"]:
        smData = d
    elif "basepower" in d["title"]:
        baseData = d
    else:
        meterData.append(d)
# Sort appliances as topmost consumers
meterData = sorted(meterData, key=lambda x: x["avgPower"], reverse=True)

NameError: name 'dataList' is not defined

In [9]:
print("")
for m in meterData:
    print("{:>33}:{:>10}W".format(m["title"], "{:.02f}".format(m["avgPower"])))
print("_"*45)
sumAvg = np.sum([m["avgPower"] for m in meterData])
print("{:>33}:{:>10}W".format("SUM", "{:.02f}".format(sumAvg)))
print("")
if baseData is not None:
    print("_"*18 + "BASE:" + "_"*22)
    print("{:>33}:{:>10}W".format(baseData["title"], "{:.02f}".format(baseData["avgPower"])))
        
if smData is not None:
    print("_"*18 + "SMARTMETER:" + "_"*16)
    print("{:>33}:{:>10}W".format(smData["title"], "{:.02f}".format(smData["avgPower"])))
    print("_"*45)
    smartMeterAvgPower = smData["avgPower"]*24/1000.0


_____________________________________________
                              SUM:      0.00W



NameError: name 'baseData' is not defined

In [10]:
topMost = 6
metersToShow = []
appliancesToShow = []
metersToHide = ["powermeter27"]

metersToPlot = []
metersRest = []

count = 0
for m in meterData:
    if count < topMost:
        if m["title"] not in metersToHide:
            metersToPlot.append(m)
        else:
            metersRest.append(m)
            count -= 1
    else:
        if m["title"] in metersToShow: metersToPlot.append(m)
        elif m["title"] in appliancesToShow: metersToPlot.append(m)
        elif m["title"] in deviceMapping and any(i for i in deviceMapping[m["title"]]["appliances"] if i.lower() in appliancesToShow):
            metersToPlot.append(m)
        else: metersRest.append(m)
    count += 1

In [11]:
replaceLabels = {"powermeter18":"Network Equipment", "powermeter20":"Office PC"}
startSample = 6*60*60

timestamps = np.linspace(start+startSample, start + duration, int(duration-startSample))
dates = [datetime.fromtimestamp(ts) for ts in timestamps]

fig, ax = plt.subplots(1,1, figsize=(7,4))

axins = ax.inset_axes([0.6, 0.4, 0.37, 0.57])


minLen = 0
def plot(ax, data, **kargs):
    global additive, last
    # Apply smoothing
    if smoothing > 1:
        data = np.median(data[0:int(len(data)/smoothing)*smoothing].reshape(-1, smoothing), axis=1) 
    # Accumulate
    additive = np.add(additive, data)
    # Plot either filled or not
    ax.fill_between(dates[::smoothing][:minLen], additive[:minLen], last[:minLen], **kargs)
    axins.fill_between(dates[::smoothing][:minLen], additive[:minLen], last[:minLen], **kargs)
    last = additive

# Plot smartmeter
data = smData["data"][measure][startSample:]
# Apply smoothing
if smoothing > 1:
    data = np.median(data[0:int(len(data)/smoothing)*smoothing].reshape(-1, smoothing), axis=1) 
minLen = len(data)
ax.plot(dates[0], 0, label="mains", lw=1, c='k')
axins.plot(dates[::smoothing][:minLen], data[:minLen], label="mains", lw=1, c='k')

additive = np.zeros(len(dates))  
if smoothing > 1:
    additive = np.median(additive[0:int(len(additive)/smoothing)*smoothing].reshape(-1, smoothing), axis=1)
last = additive
    
if baseData is not None:
    newData = baseData["data"][measure][startSample:]
    plot(ax, newData, label="Base", color='k')
    
# plot remaining data
if len(metersRest) > 0:
    newData = np.sum(np.array([pm["data"][measure][startSample:] for pm in metersRest]), axis=0)
    plot(ax, newData, label="Others")
    
# plot meters
for powermeter in reversed(metersToPlot):
    newData = powermeter["data"][measure][startSample:]
    if powermeter["title"] in replaceLabels:
        label = replaceLabels[powermeter["title"]]
    elif powermeter["title"] in deviceMapping:
        label = hp.prettyfyApplianceName(str(", ".join(deviceMapping[powermeter["title"]]["appliances"])))
    else:
        label = hp.prettyfyApplianceName(powermeter["title"])
    if len(label) > 40:
        from textwrap import wrap
        label = "\n".join(wrap(label.replace(" ", "?").replace(",", " "), width=40)).replace(" ", ",").replace("?", " ").replace(",\n", "\n").replace("\n", ",\n").replace("\n ", "\n")
    
    plot(ax, newData, label=label)

ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))


# _ = plt.xticks(rotation=45)
# Set x and y labels
ax.set_ylabel("Apparent Power [W]")
ax.set_ylim([0,2700])
ax.set_xlim([dates[0],dates[-1]])
ax.use_sticky_edges = True
ax.set_xlabel("Time of day")
ax.legend(loc="upper left")

# Oven + Espresso
zoomXStart = int(15*60*60 + 34*60)-startSample
zoomDur = int(30*60)
# Espresso morning
#zoomXStart = int(19*60*60 + 20*60)-startSample
#zoomDur = int(30*60)
# Evening
# zoomXStart = int(19*60*60 + 14*60)-startSample
# zoomDur = int(50*60)

# sub region of the original image
axins.set_xlim(dates[zoomXStart], dates[zoomXStart+zoomDur])
axins.set_ylim(0, 2750)
# axins.set_ylim(0, 480)
axins.set_xticklabels('')
axins.set_yticklabels('')
axins.set_yticks([])
axins.set_xticks([])

ax.indicate_inset_zoom(axins, lw=1.0, alpha=0.2, color='k')


NameError: name 'start' is not defined